<a href="https://colab.research.google.com/github/KevinTheRainmaker/MLOps/blob/main/%08MLOps_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 검증

In [ ]:
!pip install tensorflow-data-validation

     |████████████████████████████████| 1.4 MB 5.0 MB/s 
     |████████████████████████████████| 48 kB 3.7 MB/s 
     |████████████████████████████████| 294 kB 46.8 MB/s 
     |████████████████████████████████| 19.1 MB 337 kB/s 
     |████████████████████████████████| 9.8 MB 48.7 MB/s 
     |████████████████████████████████| 2.3 MB 38.1 MB/s 
     |████████████████████████████████| 247 kB 71.2 MB/s 
     |████████████████████████████████| 151 kB 52.0 MB/s 
     |████████████████████████████████| 62 kB 699 kB/s 
     |████████████████████████████████| 829 kB 48.2 MB/s 
     |████████████████████████████████| 171 kB 43.1 MB/s 
     |████████████████████████████████| 183 kB 41.1 MB/s 
     |████████████████████████████████| 144 kB 44.1 MB/s 
     |████████████████████████████████| 169 kB 57.6 MB/s 
     |████████████████████████████████| 255 kB 51.1 MB/s 
     |████████████████████████████████| 173 kB 57.5 MB/s 
     |████████████████████████████████| 83 kB 2.1 MB/s 
     |███████████████

In [ ]:
import tensorflow_data_validation as tfdv

# csv 로드
stats = tfdv.generate_statistics_from_csv(
	data_location='/data/consumer_complaints.csv',
    delimiter=',')

# TFRecord 로드
stats = tfdv.generate_statistics_from_tfrecord(
	data_location='/data/consumer_complaints.csv')

In [ ]:
schema = tfdv.infer_schema(stats)

In [ ]:
tfdv.display_schema(schema)

In [ ]:
# train_stats = tfdv.generate_statistics_from_tfrecord(
#     data_location=train_tfrecord_filename
# )

# val_stats = tfdv.generate_statistics_from_tfrecord(
#     data_location=val_tfrecord_filename
# )

# tfdv.visualize_statistics(lhs_statistics=val_stats, rhs_statistics=train_stats,
#                           lhs_name='VAL_DATASET', rhs_name='TRAIN_DATASET')